In [25]:
import pandas as pd
import numpy as np
import pytorch_lightning as pl

from torch import nn
from torch.nn import functional as F
from torch.utils.data import TensorDataset, DataLoader


from sklearn.model_selection import train_test_split

In [26]:
class Net(pl.LightningModule):
    def __init__(self):
        super().__init__()
        #self.conv1 = nn.Conv1d(1, 16, 4)
        #self.pool = nn.MaxPool1d(2)
        self.fc1 = nn.Linear(187, 1)
    
    def forward(self, x):
        x = self.fc1(x)

        return x

    def training_step(self, batch, batch_idx):
        x, y = batch
        y_hat = self(x)
        loss = F.binary_cross_entropy(y_hat, y)
        return loss

    def validation_step(self, batch, batch_idx):
        x, y = batch
        y_hat = self.model(x)
        loss = F.binary_cross_entropy(y_hat, y)
        return loss


    def configure_optimizers(self):
        return torch.optim.Adam(self.parameters(), lr = 0.001)

In [27]:
class NetDataset(pl.LightningDataModule):
    def __init__(self, batch_size=64):
        super().__init__()
        self.batch_size = batch_size
    
    def setup(self, stage=None):
        df_1= pd.read_csv("../data/ptbdb_normal.csv", header=None)
        df_2 = pd.read_csv("../data/ptbdb_abnormal.csv", header=None)
        df = pd.concat([df_1, df_2])

        df_train, df_test = train_test_split(df, test_size=0.2, random_state=1337, stratify=df[187])

        Y_train = torch.from_numpy(np.array(df_train[187].values).astype(np.int8))
        X_train = torch.from_numpy(np.array(df_train[list(range(187))].values)[..., np.newaxis])

        Y_test = torch.from_numpy(np.array(df_test[187].values).astype(np.int8))
        X_test = torch.from_numpy(np.array(df_test[list(range(187))].values)[..., np.newaxis])

        self.train_dataset = TensorDataset(X_train, Y_train)
        self.test_dataset = TensorDataset(X_test, Y_test)

    def train_dataloader(self):
        train_loader = DataLoader(self.train_dataset)
        return train_loader
    
    def test_dataloader(self):
        test_loader = DataLoader(self.test_dataset)
        return test_loader

In [32]:
model = Net()
dataset = NetDataset()
trainer = pl.Trainer()
trainer.fit(model=model, datamodule=dataset)

GPU available: False, used: False
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
/home/julian/miniconda3/envs/ml4h/lib/python3.9/site-packages/pytorch_lightning/trainer/configuration_validator.py:122: UserWarning: You defined a `validation_step` but have no `val_dataloader`. Skipping val loop.
  rank_zero_warn("You defined a `validation_step` but have no `val_dataloader`. Skipping val loop.")


TypeError: setup() got an unexpected keyword argument 'stage'